<a href="https://colab.research.google.com/github/steffenmodest/notebooks/blob/master/patent_neuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Helper function to encode XOR input as per patent description
# le1, le2 are the logical inputs (0 or 1)
# Returns a 4-element numpy array [e1, e2, e3, e4]
# e1 active if le1=0, e2 active if le1=1
# e3 active if le2=0, e4 active if le2=1
def encode_xor_input(le1, le2):
    e = np.zeros(4)
    if le1 == 0:
        e[0] = 1 # e1 active
    else:
        e[1] = 1 # e2 active
    if le2 == 0:
        e[2] = 1 # e3 active
    else:
        e[3] = 1 # e4 active
    return e

class NeuronWithMemory:
    """
    Implements a neuron based on DE 43 37 603 A1 with memory devices (SVs).
    Uses the Delta Rule for base weight learning.
    """
    def __init__(self, num_inputs, learning_rate=0.1, threshold=1.0):
        # Initialize base weights (Synapsen 2) - starting near zero
        self.weights = np.random.uniform(-0.1, 0.1, num_inputs)
        # self.weights = np.zeros(num_inputs) # Alternative: start at zero like patent table implicitly
        self.threshold = threshold # Schwellenwert Theta
        self.learning_rate = learning_rate
        # List to store memory devices (Speichervorrichtungen 3)
        # Each SV stores the pattern of *active input indices* and the modification type
        self.memory_devices = [] # Format: [{'pattern': (indices_tuple), 'type': 'increase'/'decrease'}]

    def _activate(self, net_input):
        """Applies the threshold activation function (f(net - Theta))"""
        # f(x)=1 if x >= 0, f(x) = 0 if x < 0 (as per patent logic net >= Theta -> 1)
        return 1 if net_input >= self.threshold else 0

    def _get_active_input_indices(self, inputs):
        """Returns a tuple of indices where the input is active (e.g., == 1)"""
        return tuple(np.where(inputs == 1)[0])

    def _calculate_effective_weights(self, inputs):
        """
        Calculates the momentary effective weights, applying SV modifications.
        """
        active_indices_set = set(self._get_active_input_indices(inputs))
        effective_weights = self.weights.copy() # Start with base weights
        triggered_sv = False

        # Check if any memory device matches the current active input pattern
        for sv in self.memory_devices:
            sv_pattern_set = set(sv['pattern'])
            # Check for exact match of active inputs
            if active_indices_set == sv_pattern_set:
                print(f"  --> SV Triggered: Pattern {sv['pattern']}, Type: {sv['type']}")
                # Apply modification based on SV type
                modification_factor = 2.0 if sv['type'] == 'increase' else 0.5
                for index in sv['pattern']:
                    effective_weights[index] *= modification_factor
                triggered_sv = True
                break # Assume only one SV triggers per step for simplicity

        # Debug print if an SV was triggered
        # if triggered_sv:
        #    print(f"  Base weights: {self.weights}")
        #    print(f"  Effective weights: {effective_weights}")

        return effective_weights

    def predict(self, inputs):
        """
        Predicts the output for given inputs, considering memory devices.
        """
        # 1. Calculate effective weights based on potential SV trigger
        effective_weights = self._calculate_effective_weights(inputs)

        # 2. Calculate net input using *effective* weights
        # net = S w_j * e_j (Formel 1 from patent)
        net_input = np.dot(effective_weights, inputs)

        # 3. Apply activation function
        output = self._activate(net_input)
        return output, net_input # Return net_input for Delta Rule calculation

    def _add_memory(self, inputs, error_type):
        """
        Adds a memory device (SV) if an error occurred for this input pattern.
        error_type: 'increase' (output too low) or 'decrease' (output too high)
        """
        pattern_indices = self._get_active_input_indices(inputs)

        # Check if a memory for this exact pattern already exists
        pattern_exists = any(set(sv['pattern']) == set(pattern_indices) for sv in self.memory_devices)

        if not pattern_exists and pattern_indices: # Avoid storing empty patterns
            print(f"  --> Adding SV: Pattern {pattern_indices}, Type: {error_type}")
            self.memory_devices.append({'pattern': pattern_indices, 'type': error_type})
        # else:
            # print(f"  --> SV for pattern {pattern_indices} already exists or pattern empty.")


    def train_step(self, inputs, target):
        """
        Performs a single training step (prediction and weight update).
        """
        # 1. Predict output using current state (incl. SVs)
        prediction, net_input = self.predict(inputs)
        error = target - prediction

        print(f"Input: {inputs}, Target: {target}, Pred: {prediction}, Net: {net_input:.2f}, Error: {error}")

        # 2. Update base weights using Delta Rule if there is an error
        # Formula: delta_w = learning_rate * error * input
        # Note: Some delta rule versions use (target - net_input) instead of (target - prediction)
        # Let's use the simpler (target - prediction) version which aligns with Perceptron rule logic
        if error != 0:
            delta_w = self.learning_rate * error * inputs
            self.weights += delta_w
            print(f"  Base weights updated by: {delta_w}")
            print(f"  New base weights: {self.weights}")

            # 3. Add a memory device (SV) for the problematic pattern
            # Determine if the effect should be increased (output too low) or decreased (output too high)
            if error > 0: # Target was 1, prediction was 0 -> output too low
                self._add_memory(inputs, 'increase')
            else: # Target was 0, prediction was 1 -> output too high
                self._add_memory(inputs, 'decrease')
        # else: No error, no weight update, no new memory needed


# --- Training Data (XOR) ---
# Using the encoding from the patent: [e1, e2, e3, e4]
xor_data = [
    # le1, le2, target
    (0, 0, 0),
    (1, 0, 1),
    (0, 1, 1),
    (1, 1, 0),
]

# Encode the data
encoded_xor_inputs = np.array([encode_xor_input(le1, le2) for le1, le2, _ in xor_data])
xor_targets = np.array([target for _, _, target in xor_data])

# --- Initialize Neuron ---
num_inputs = 4
neuron = NeuronWithMemory(num_inputs=num_inputs, learning_rate=0.5, threshold=1.0) # LR=0.5 like patent example start

print("--- Initial State ---")
print(f"Base Weights (w1-w4): {neuron.weights}")
print(f"Memory Devices (SVs): {neuron.memory_devices}")
print(f"Threshold (Theta): {neuron.threshold}")
print("-" * 25)

# --- Training Loop ---
epochs = 10
print(f"--- Starting Training ({epochs} epochs) ---")
for epoch in range(epochs):
    print(f"\n--- Epoch {epoch + 1} ---")
    correct_predictions = 0
    for i in range(len(encoded_xor_inputs)):
        inputs = encoded_xor_inputs[i]
        target = xor_targets[i]
        print(f"Training sample {i} (Logical: {xor_data[i][:2]} -> {target})")
        neuron.train_step(inputs, target)
        # Check prediction *after* potential update for accuracy count
        final_pred, _ = neuron.predict(inputs)
        if final_pred == target:
            correct_predictions += 1

    print(f"Epoch {epoch + 1} Accuracy: {correct_predictions}/{len(encoded_xor_inputs)}")
    print(f"Current Base Weights: {neuron.weights}")
    print(f"Current Memory Devices: {neuron.memory_devices}")
    if correct_predictions == len(encoded_xor_inputs):
        print("\nTraining converged!")
       # break # Optional: stop early if converged

print("\n--- Training Finished ---")

# --- Test Final Neuron ---
print("\n--- Final Test ---")
print(f"Final Base Weights (w1-w4): {neuron.weights}")
print(f"Final Memory Devices (SVs): {neuron.memory_devices}")
for i in range(len(encoded_xor_inputs)):
    inputs = encoded_xor_inputs[i]
    target = xor_targets[i]
    prediction, net_input = neuron.predict(inputs)
    print(f"Input: {inputs} (Logical: {xor_data[i][:2]}), Target: {target}, Prediction: {prediction}")

--- Initial State ---
Base Weights (w1-w4): [-0.00475667 -0.00792924  0.04318812  0.09015639]
Memory Devices (SVs): []
Threshold (Theta): 1.0
-------------------------
--- Starting Training (10 epochs) ---

--- Epoch 1 ---
Training sample 0 (Logical: (0, 0) -> 0)
Input: [1. 0. 1. 0.], Target: 0, Pred: 0, Net: 0.04, Error: 0
Training sample 1 (Logical: (1, 0) -> 1)
Input: [0. 1. 1. 0.], Target: 1, Pred: 0, Net: 0.04, Error: 1
  Base weights updated by: [0.  0.5 0.5 0. ]
  New base weights: [-0.00475667  0.49207076  0.54318812  0.09015639]
  --> Adding SV: Pattern (np.int64(1), np.int64(2)), Type: increase
  --> SV Triggered: Pattern (np.int64(1), np.int64(2)), Type: increase
Training sample 2 (Logical: (0, 1) -> 1)
Input: [1. 0. 0. 1.], Target: 1, Pred: 0, Net: 0.09, Error: 1
  Base weights updated by: [0.5 0.  0.  0.5]
  New base weights: [0.49524333 0.49207076 0.54318812 0.59015639]
  --> Adding SV: Pattern (np.int64(0), np.int64(3)), Type: increase
  --> SV Triggered: Pattern (np.int

In [5]:
import numpy as np
import time # To slightly vary random seed if run quickly

# Helper function to encode XOR input as per patent description
# le1, le2 are the logical inputs (0 or 1)
# Returns a 4-element numpy array [e1, e2, e3, e4]
# e1 active if le1=0, e2 active if le1=1
# e3 active if le2=0, e4 active if le2=1
def encode_xor_input(le1, le2):
    e = np.zeros(4)
    if le1 == 0:
        e[0] = 1 # e1 active
    else:
        e[1] = 1 # e2 active
    if le2 == 0:
        e[2] = 1 # e3 active
    else:
        e[3] = 1 # e4 active
    return e

class NeuronWithMemoryClaim4:
    """
    Implements a neuron based on DE 43 37 603 A1 with memory devices (SVs).
    Uses the Delta Rule for base weight learning.
    Implements the restriction from Claim 4.
    """
    def __init__(self, num_inputs, learning_rate=0.1, threshold=1.0):
        # Initialize base weights (Synapsen 2) - starting near zero
        # Adding a time-based component to seed for potentially different runs
        np.random.seed(int(time.time() * 1000) % (2**32))
        self.weights = np.random.uniform(-0.1, 0.1, num_inputs)
        # self.weights = np.zeros(num_inputs) # Alternative: start at zero
        self.threshold = threshold # Schwellenwert Theta
        self.learning_rate = learning_rate
        # List to store memory devices (Speichervorrichtungen 3)
        # Format: [{'pattern': (indices_tuple), 'type': 'increase'/'decrease'}]
        self.memory_devices = []
        # Set to keep track of individual input indices already claimed by an SV (for Claim 4)
        self.claimed_input_indices = set()

    def _activate(self, net_input):
        """Applies the threshold activation function (f(net - Theta))"""
        return 1 if net_input >= self.threshold else 0

    def _get_active_input_indices(self, inputs):
        """Returns a tuple of indices where the input is active (e.g., == 1)"""
        return tuple(np.where(inputs == 1)[0])

    def _calculate_effective_weights(self, inputs):
        """
        Calculates the momentary effective weights, applying SV modifications.
        """
        active_indices_set = set(self._get_active_input_indices(inputs))
        effective_weights = self.weights.copy() # Start with base weights
        triggered_sv = False

        # Check if any memory device matches the current active input pattern
        for sv in self.memory_devices:
            sv_pattern_set = set(sv['pattern'])
            # Check for exact match of active inputs
            if active_indices_set == sv_pattern_set:
                print(f"  --> SV Triggered: Pattern {sv['pattern']}, Type: {sv['type']}")
                # Apply modification based on SV type
                modification_factor = 2.0 if sv['type'] == 'increase' else 0.5
                for index in sv['pattern']:
                     # Ensure index is within bounds before modifying
                    if 0 <= index < len(effective_weights):
                        effective_weights[index] *= modification_factor
                    else:
                        print(f"  Warning: Index {index} out of bounds for weights.")

                triggered_sv = True
                break # Assume only one SV triggers per step

        return effective_weights

    def predict(self, inputs):
        """
        Predicts the output for given inputs, considering memory devices.
        """
        effective_weights = self._calculate_effective_weights(inputs)
        net_input = np.dot(effective_weights, inputs)
        output = self._activate(net_input)
        return output, net_input

    def _add_memory(self, inputs, error_type):
        """
        Adds a memory device (SV) if an error occurred, respecting Claim 4.
        error_type: 'increase' (output too low) or 'decrease' (output too high)
        """
        pattern_indices = self._get_active_input_indices(inputs)

        # Condition 0: Don't store empty patterns
        if not pattern_indices:
            # print("  --> SV not added: Pattern is empty.")
            return

        # Condition 1: Check if the exact same pattern already exists
        pattern_exists = any(set(sv['pattern']) == set(pattern_indices) for sv in self.memory_devices)
        if pattern_exists:
            # print(f"  --> SV not added: Pattern {pattern_indices} already exists.")
            return

        # Condition 2: Implement Claim 4 rule
        # Check if *any* input index in the new pattern is *already* claimed by *any* existing SV.
        overlap = False
        claimed_by = set()
        for index in pattern_indices:
            if index in self.claimed_input_indices:
                overlap = True
                claimed_by.add(index)
                # No need to break here, collect all overlapping indices for the message

        if overlap:
            print(f"  --> SV not added for {pattern_indices}: Input(s) {claimed_by} already claimed by other SVs (Claim 4).")
            return

        # If all checks passed, add the new SV
        print(f"  --> Adding SV: Pattern {pattern_indices}, Type: {error_type}")
        self.memory_devices.append({'pattern': pattern_indices, 'type': error_type})
        # --- IMPORTANT: Update the set of claimed indices ---
        self.claimed_input_indices.update(pattern_indices)
        print(f"  --> Claimed indices updated: {self.claimed_input_indices}")


    def train_step(self, inputs, target):
        """
        Performs a single training step (prediction and weight update).
        """
        prediction, net_input = self.predict(inputs)
        error = target - prediction

        print(f"Input: {inputs}, Target: {target}, Pred: {prediction}, Net: {net_input:.2f}, Error: {error}")

        if error != 0:
            # Update base weights using Delta Rule
            delta_w = self.learning_rate * error * inputs
            self.weights += delta_w
            print(f"  Base weights updated by: {delta_w}")
            # print(f"  New base weights: {self.weights}") # Verbose print

            # Attempt to add a memory device (SV)
            if error > 0: # Target=1, Pred=0 -> increase effect
                self._add_memory(inputs, 'increase')
            else: # Target=0, Pred=1 -> decrease effect
                self._add_memory(inputs, 'decrease')


# --- Training Data (XOR) ---
xor_data = [ (0, 0, 0), (1, 0, 1), (0, 1, 1), (1, 1, 0) ]
encoded_xor_inputs = np.array([encode_xor_input(le1, le2) for le1, le2, _ in xor_data])
xor_targets = np.array([target for _, _, target in xor_data])

# --- Initialize Neuron ---
num_inputs = 4
# Using higher learning rate and specific threshold from patent example context
neuron = NeuronWithMemoryClaim4(num_inputs=num_inputs, learning_rate=0.5, threshold=1.0)

print("--- Initial State (Claim 4 Neuron) ---")
print(f"Initial Base Weights (w1-w4): {neuron.weights}")
print(f"Memory Devices (SVs): {neuron.memory_devices}")
print(f"Claimed Indices: {neuron.claimed_input_indices}")
print(f"Threshold (Theta): {neuron.threshold}")
print("-" * 35)

# --- Training Loop ---
epochs = 15 # May need more epochs or different LR due to Claim 4 restriction
print(f"--- Starting Training ({epochs} epochs) ---")
converged_epoch = -1
for epoch in range(epochs):
    print(f"\n--- Epoch {epoch + 1} ---")
    correct_predictions = 0
    # Shuffle data order each epoch for potentially different learning dynamics
    indices = np.arange(len(encoded_xor_inputs))
    np.random.shuffle(indices)

    for i in indices:
        inputs = encoded_xor_inputs[i]
        target = xor_targets[i]
        print(f"Training sample index {i} (Logical: {xor_data[i][:2]} -> {target})")
        neuron.train_step(inputs, target)
        # Check prediction *after* potential update for accuracy count
        final_pred, _ = neuron.predict(inputs)
        if final_pred == target:
            correct_predictions += 1

    print(f"Epoch {epoch + 1} Accuracy: {correct_predictions}/{len(encoded_xor_inputs)}")
    print(f"Current Base Weights: {neuron.weights}")
    print(f"Current Memory Devices: {neuron.memory_devices}")
    print(f"Currently Claimed Indices: {neuron.claimed_input_indices}")
    if correct_predictions == len(encoded_xor_inputs) and converged_epoch == -1:
        print("\nTraining converged!")
        converged_epoch = epoch + 1
        # Let it run a bit longer to see if state remains stable
        # break # Optional: stop early

print(f"\n--- Training Finished (Converged in epoch: {converged_epoch if converged_epoch != -1 else 'Not converged'}) ---")

# --- Test Final Neuron ---
print("\n--- Final Test (Claim 4 Neuron) ---")
print(f"Final Base Weights (w1-w4): {neuron.weights}")
print(f"Final Memory Devices (SVs): {neuron.memory_devices}")
print(f"Final Claimed Indices: {neuron.claimed_input_indices}")
correct_final = 0
for i in range(len(encoded_xor_inputs)):
    inputs = encoded_xor_inputs[i]
    target = xor_targets[i]
    prediction, net_input = neuron.predict(inputs)
    print(f"Input: {inputs} (Logical: {xor_data[i][:2]}), Target: {target}, Prediction: {prediction}")
    if prediction == target:
        correct_final +=1
print(f"Final Accuracy on Training Data: {correct_final}/{len(encoded_xor_inputs)}")

--- Initial State (Claim 4 Neuron) ---
Initial Base Weights (w1-w4): [ 0.06448819  0.08254    -0.08802165 -0.08650971]
Memory Devices (SVs): []
Claimed Indices: set()
Threshold (Theta): 1.0
-----------------------------------
--- Starting Training (15 epochs) ---

--- Epoch 1 ---
Training sample index 2 (Logical: (0, 1) -> 1)
Input: [1. 0. 0. 1.], Target: 1, Pred: 0, Net: -0.02, Error: 1
  Base weights updated by: [0.5 0.  0.  0.5]
  --> Adding SV: Pattern (np.int64(0), np.int64(3)), Type: increase
  --> Claimed indices updated: {np.int64(0), np.int64(3)}
  --> SV Triggered: Pattern (np.int64(0), np.int64(3)), Type: increase
Training sample index 0 (Logical: (0, 0) -> 0)
Input: [1. 0. 1. 0.], Target: 0, Pred: 0, Net: 0.48, Error: 0
Training sample index 1 (Logical: (1, 0) -> 1)
Input: [0. 1. 1. 0.], Target: 1, Pred: 0, Net: -0.01, Error: 1
  Base weights updated by: [0.  0.5 0.5 0. ]
  --> Adding SV: Pattern (np.int64(1), np.int64(2)), Type: increase
  --> Claimed indices updated: {np.

In [11]:
import numpy as np
import time

# No input encoding needed for random binary data

class NeuronWithMemoryClaim4:
    """
    Implements a neuron based on DE 43 37 603 A1 with memory devices (SVs).
    Uses the Delta Rule for base weight learning.
    Implements the restriction from Claim 4.
    (Unchanged from previous version)
    """
    def __init__(self, num_inputs, learning_rate=0.1, threshold=1.0):
        # Initialize base weights (Synapsen 2) - starting near zero
        # REMOVED the problematic np.random.seed() line.
        # np.random.uniform will use NumPy's internal random state.
        self.weights = np.random.uniform(-0.1, 0.1, num_inputs)
        self.threshold = threshold # Schwellenwert Theta
        self.learning_rate = learning_rate
        # List to store memory devices (Speichervorrichtungen 3)
        self.memory_devices = [] # Format: [{'pattern': (indices_tuple), 'type': 'increase'/'decrease'}]
        # Set to keep track of individual input indices claimed by this neuron's SVs
        self.claimed_input_indices = set()

    def _activate(self, net_input):
        """Applies the threshold activation function (f(net - Theta))"""
        return 1 if net_input >= self.threshold else 0

    def _get_active_input_indices(self, inputs):
        """Returns a tuple of indices where the input is active (e.g., == 1)"""
        # Ensure inputs is treated as a flat array for np.where
        return tuple(np.where(np.asarray(inputs).flatten() == 1)[0])

    def _calculate_effective_weights(self, inputs):
        """
        Calculates the momentary effective weights, applying SV modifications.
        """
        active_indices_set = set(self._get_active_input_indices(inputs))
        effective_weights = self.weights.copy() # Start with base weights
        triggered_sv = False

        for sv in self.memory_devices:
            sv_pattern_set = set(sv['pattern'])
            if active_indices_set == sv_pattern_set:
                # --- Debug print moved inside the if for clarity ---
                # print(f"  --> SV Triggered: Neuron {id(self)}, Pattern {sv['pattern']}, Type: {sv['type']}")
                modification_factor = 2.0 if sv['type'] == 'increase' else 0.5
                for index in sv['pattern']:
                    if 0 <= index < len(effective_weights):
                        effective_weights[index] *= modification_factor
                    else:
                        print(f"  Warning: Index {index} out of bounds for weights.")
                triggered_sv = True
                break

        return effective_weights

    def predict(self, inputs):
        """
        Predicts the output for given inputs, considering memory devices.
        Returns prediction (0 or 1) and raw net input.
        """
        effective_weights = self._calculate_effective_weights(inputs)
        # Ensure inputs is treated as a flat array for dot product
        net_input = np.dot(effective_weights, np.asarray(inputs).flatten())
        output = self._activate(net_input)
        return output, net_input

    def _add_memory(self, inputs, error_type):
        """
        Adds a memory device (SV) if an error occurred, respecting Claim 4.
        """
        pattern_indices = self._get_active_input_indices(inputs)

        if not pattern_indices:
            return

        pattern_exists = any(set(sv['pattern']) == set(pattern_indices) for sv in self.memory_devices)
        if pattern_exists:
            return

        # Check Claim 4 rule: overlap with already claimed indices for *this* neuron
        overlap = False
        claimed_by = set()
        for index in pattern_indices:
            if index in self.claimed_input_indices:
                overlap = True
                claimed_by.add(index)

        if overlap:
            # print(f"  --> Neuron {id(self)}: SV not added for {pattern_indices} due to Claim 4 (Overlap: {claimed_by}).")
            return

        # Add the new SV
        # print(f"  --> Neuron {id(self)}: Adding SV: Pattern {pattern_indices}, Type: {error_type}")
        self.memory_devices.append({'pattern': pattern_indices, 'type': error_type})
        self.claimed_input_indices.update(pattern_indices)
        # print(f"  --> Neuron {id(self)}: Claimed indices updated: {self.claimed_input_indices}")

    def train_step(self, inputs, target, neuron_id_str=""):
        """
        Performs a single training step for this neuron.
        """
        prediction, net_input = self.predict(inputs)
        error = target - prediction

        # print(f"Neuron {neuron_id_str} - Input: {inputs}, Target: {target}, Pred: {prediction}, Net: {net_input:.2f}, Error: {error}")

        if error != 0:
            # Update base weights using Delta Rule
            delta_w = self.learning_rate * error * np.asarray(inputs).flatten()
            self.weights += delta_w
            # print(f"  Neuron {neuron_id_str} - Base weights updated by: {delta_w}")

            # Attempt to add a memory device (SV)
            if error > 0: # Target=1, Pred=0 -> increase effect
                self._add_memory(inputs, 'increase')
            else: # Target=0, Pred=1 -> decrease effect
                self._add_memory(inputs, 'decrease')

# --- Data Generation ---
def generate_random_binary_data(num_samples, num_inputs, num_outputs):
    """Generates random binary input and target data."""
    # Ensure reproducibility for a single run if needed, but allow randomness across runs
    # np.random.seed(42) # Set seed if deterministic generation is required
    inputs = np.random.randint(0, 2, size=(num_samples, num_inputs))
    targets = np.random.randint(0, 2, size=(num_samples, num_outputs))
    return inputs, targets

# --- Network Evaluation ---
def evaluate_network(network, inputs, targets):
    """Calculates the accuracy of the network on the given data."""
    correct_predictions = 0
    num_samples = len(inputs)
    if num_samples == 0:
        return 0.0

    for i in range(num_samples):
        input_vec = inputs[i]
        target_vec = targets[i]
        predictions = []
        for neuron in network:
            pred, _ = neuron.predict(input_vec)
            predictions.append(pred)

        # Check if *all* predictions for the sample match the targets
        if np.array_equal(predictions, target_vec):
            correct_predictions += 1

    accuracy = correct_predictions / num_samples
    return accuracy

# --- Parameters ---
NUM_INPUTS = 5
NUM_OUTPUTS = 2
NUM_TRAIN_SAMPLES = 100
NUM_TEST_SAMPLES = 50
EPOCHS = 50
LEARNING_RATE = 0.1 # Lowered learning rate often better for more complex/random data
THRESHOLD = 1.0     # Threshold might need tuning depending on data distribution

# --- Generate Data ---
train_inputs, train_targets = generate_random_binary_data(NUM_TRAIN_SAMPLES, NUM_INPUTS, NUM_OUTPUTS)
test_inputs, test_targets = generate_random_binary_data(NUM_TEST_SAMPLES, NUM_INPUTS, NUM_OUTPUTS)

print(f"--- Data Generated ---")
print(f"Training samples: {NUM_TRAIN_SAMPLES}")
print(f"Test samples: {NUM_TEST_SAMPLES}")
print(f"Input features: {NUM_INPUTS}")
print(f"Output features: {NUM_OUTPUTS}")
print("-" * 30)

# --- Initialize Network (One neuron per output) ---
network = [NeuronWithMemoryClaim4(num_inputs=NUM_INPUTS, learning_rate=LEARNING_RATE, threshold=THRESHOLD)
           for _ in range(NUM_OUTPUTS)]

print(f"--- Network Initialized ({len(network)} neurons) ---")
for i, neuron in enumerate(network):
    print(f"Neuron {i} Initial Weights: {neuron.weights}")
print("-" * 30)


# --- Training Loop ---
print(f"--- Starting Training ({EPOCHS} epochs) ---")
for epoch in range(EPOCHS):
    # --- Train one epoch ---
    # Shuffle training data order each epoch
    indices = np.arange(NUM_TRAIN_SAMPLES)
    np.random.shuffle(indices)

    for i in indices:
        input_vec = train_inputs[i]
        target_vec = train_targets[i]
        # Train each neuron in the network on its corresponding target
        for neuron_idx, neuron in enumerate(network):
            neuron.train_step(input_vec, target_vec[neuron_idx], neuron_id_str=str(neuron_idx))

    # --- Evaluate after epoch ---
    train_accuracy = evaluate_network(network, train_inputs, train_targets)
    test_accuracy = evaluate_network(network, test_inputs, test_targets)

    print(f"Epoch {epoch + 1}/{EPOCHS} - Train Accuracy: {train_accuracy:.4f} - Test Accuracy: {test_accuracy:.4f}")

    # Optional: Print SV status for the first neuron occasionally
    if (epoch + 1) % 10 == 0:
         print(f"  Neuron 0 SVs: {len(network[0].memory_devices)}, Claimed Indices: {network[0].claimed_input_indices}")


print("\n--- Training Finished ---")

# --- Final Evaluation ---
final_train_accuracy = evaluate_network(network, train_inputs, train_targets)
final_test_accuracy = evaluate_network(network, test_inputs, test_targets)

print("\n--- Final Evaluation ---")
print(f"Final Training Accuracy: {final_train_accuracy:.4f}")
print(f"Final Test Accuracy:     {final_test_accuracy:.4f}")

print("\n--- Final Neuron States ---")
for i, neuron in enumerate(network):
    print(f"Neuron {i}:")
    print(f"  Final Base Weights: {neuron.weights}")
    print(f"  Number of SVs: {len(neuron.memory_devices)}")
    # print(f"  SVs: {neuron.memory_devices}") # Can be very verbose
    print(f"  Claimed Indices: {neuron.claimed_input_indices}")

--- Data Generated ---
Training samples: 100
Test samples: 50
Input features: 5
Output features: 2
------------------------------
--- Network Initialized (2 neurons) ---
Neuron 0 Initial Weights: [ 0.05398783 -0.08527081  0.04545925  0.08953458  0.09059758]
Neuron 1 Initial Weights: [-0.09329448 -0.01918902  0.07293938 -0.04468505 -0.08621658]
------------------------------
--- Starting Training (50 epochs) ---
Epoch 1/50 - Train Accuracy: 0.2100 - Test Accuracy: 0.2400
Epoch 2/50 - Train Accuracy: 0.2400 - Test Accuracy: 0.2600
Epoch 3/50 - Train Accuracy: 0.2500 - Test Accuracy: 0.2600
Epoch 4/50 - Train Accuracy: 0.2100 - Test Accuracy: 0.3200
Epoch 5/50 - Train Accuracy: 0.2200 - Test Accuracy: 0.2600
Epoch 6/50 - Train Accuracy: 0.2500 - Test Accuracy: 0.2800
Epoch 7/50 - Train Accuracy: 0.2300 - Test Accuracy: 0.2400
Epoch 8/50 - Train Accuracy: 0.2500 - Test Accuracy: 0.2200
Epoch 9/50 - Train Accuracy: 0.2400 - Test Accuracy: 0.2800
Epoch 10/50 - Train Accuracy: 0.2800 - Test A